In [27]:
import sys
!{sys.executable} -m pip install qdrant-client ollama sqlalchemy

  Using cached sqlalchemy-2.0.44-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.5 kB)
Using cached sqlalchemy-2.0.44-cp313-cp313-macosx_11_0_arm64.whl (2.1 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [28]:
import os
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(os.path.join(os.getcwd(),'input/meggle-food-ingredients.csv'), delimiter=';')
# Strip whitespace from string columns
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df.head()

,product_name,product_description,shelflife,sample_weights,CN_code,country_of_origin,product_group,product_group_new,product_family,product_family_internal,...,rapeseed_fat,hydrogenated_fat,non_hydrogenated_fat,fat_content_lt_40,fat_content_gt_40_lt_70,70-80,0-20,20-50,50-70,70-95
0,COMBIMEC N,High functional milk protein compound with whe...,18 months,NaN,1901 9099,DE,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,MTM® E 70,"Protein ~71%, increase of dry matter, stirred ...",18 months,NaN,2106 9098,DE,Milk Protein Compound,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,True,False
2,COMBIMEC A,High functional milk protein compound with whe...,18 months,NaN,1901 9099,DE,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,COMBIMEC TA,High functional milk protein compound with whe...,18 months,NaN,1901 9099,DE,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,Megglite® 60-02 H,"Hydrogenated dairy fat >60%, low free fat cont...",18 months,NaN,2106 9098,DE,Coffee Whitener,NaN,NaN,NaN,...,False,True,False,False,True,False,False,False,False,False


In [ ]:
from sqlalchemy import create_engine, text

# Create an in-memory SQLite database and store the DataFrame as a table
engine = create_engine('sqlite://', echo=False)
df.to_sql('meggle_food_ingredients', con=engine, index=False)

with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM meggle_food_ingredients LIMIT 5"))
    for row in result:
        print(row)


('COMBIMEC N', 'High functional milk protein compound with whey protein and caseinate, customer exclusive product! neutral pH value (6-7), stabilization of fermented milk products and cream cheese', '18 months', None, '1901 9099', 'DE', None, None, None, None, 'dairy,bakery & sweets,savoury', None, 1, 1, 0, 1, 0, 1, 0, 'FALSE', '', '', '', '', 'not vegetarian', 'not vegan', 'not palmfree', 0, '', 0, 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
('MTM® E 70', 'Protein ~71%, increase of dry matter, stirred yoghurt', '18 months', None, '2106 9098', 'DE', 'Milk Protein Compound', None, None, None, 'dairy', None, 1, 0, 0, 0, 0, 1, 0, 'FALSE', '', '', '', '', 'not vegetarian', 'not vegan', 'not palmfree', 1, '', 0, 0, '', 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)
('COMBIMEC A', 'High functional milk protein compound with whey protein and caseinate, customer exclusive product! low pH value (3,5-4), stabilization of fermented milk products and cream cheese', '18 months', None, '1901 9099', 'DE', None, None, None, 

In [32]:
# Generate schema information in JSON format
schema_info_json = {}

for column in df.columns:
    schema_info_json[column] = {
        'dtype': df[column].dtype,
        'non_null_count': df[column].count(),
        'null_count': df[column].isnull().sum(),
        'unique_values': df[column].nunique(),
        'sample_values': df[column].dropna().unique()[:5].tolist(),  # Beispielwerte (max. 5)
        'statistics': df[column].describe().to_dict() if pd.api.types.is_numeric_dtype(df[column]) else None
    }

# print(schema_info_json)
sample_rows = df.sample(2, random_state=1)

content = {'columns': schema_info_json, 'sample_rows': sample_rows.to_dict(orient='records')}
content_json = str(content).replace("'", '"').replace('nan', 'null')

d = {'object_type': 'table', 'table_name': 'products', 'json': content_json}
df_knowledge = pd.DataFrame([d])
# df_knowledge = pd.json_normalize(df_knowledge)
# df_knowledge.T
print(df_knowledge['json'].iloc[0][1:1000])

"columns": {"product_name": {"dtype": dtype("O"), "non_null_count": np.int64(137), "null_count": np.int64(9), "unique_values": 136, "sample_values": ["COMBIMEC N", "MTM® E 70", "COMBIMEC A", "COMBIMEC TA", "Megglite® 60-02 H"], "statistics": None}, "product_description": {"dtype": dtype("O"), "non_null_count": np.int64(122), "null_count": np.int64(24), "unique_values": 96, "sample_values": ["High functional milk protein compound with whey protein and caseinate, customer exclusive product! neutral pH value (6-7), stabilization of fermented milk products and cream cheese", "Protein ~71%, increase of dry matter, stirred yoghurt", "High functional milk protein compound with whey protein and caseinate, customer exclusive product! low pH value (3,5-4), stabilization of fermented milk products and cream cheese", "Hydrogenated dairy fat >60%, low free fat content, without beta carotene, with lactose", "Protein ~70%, increase of dry matter, stirred yoghurt"], "statistics": None}, "shelflife": 


In [33]:
# Select the relevant columns for the query
selected_columns = ['product_name', 'product_description', 'product_group', 'product_segment_text', 'palmfree', 'vegan', 'vegetarian']

df_qrs = df[selected_columns].copy()
# df_qrs = df_qrs.rename(columns={'Extract embeddings from page content': 'embeddings'})
# df_qrs['Adresse'] = df_qrs['Adresse'].astype(str)

# Clean up the DataFrame by dropping rows with NaN values
df_qrs.dropna(inplace=True)

# Strip whitespace from string columns
df_qrs = df_qrs.map(lambda x: x.strip() if isinstance(x, str) else x)

# Convert the DataFrame to a JSON string for each row
for i, row in df_qrs.iterrows():
    # Convert the product_segments string to a list of floats
    df_qrs.at[i, 'json'] = row.to_json()

# print the first few rows of the cleaned DataFrame
df_qrs

,product_name,product_description,product_group,product_segment_text,palmfree,vegan,vegetarian,json
1,MTM® E 70,"Protein ~71%, increase of dry matter, stirred ...",Milk Protein Compound,dairy,not palmfree,not vegan,not vegetarian,"{""product_name"":""MTM\u00ae E 70"",""product_desc..."
4,Megglite® 60-02 H,"Hydrogenated dairy fat >60%, low free fat cont...",Coffee Whitener,bakery & sweets,palmfree,not vegan,not vegetarian,"{""product_name"":""Megglite\u00ae 60-02 H"",""prod..."
5,MTM® T 80,"Protein ~70%, increase of dry matter, stirred ...",Milk Protein Compound,dairy,palmfree,not vegan,not vegetarian,"{""product_name"":""MTM\u00ae T 80"",""product_desc..."
6,Megglite® 30-07 MB,"Hydrogenated dairy fat >28%, low free fat cont...",Coffee Whitener,bakery & sweets,not palmfree,not vegan,vegetarian,"{""product_name"":""Megglite\u00ae 30-07 MB"",""pro..."
7,Megglite® 35-08,"Hydrogenated dairy fat >32%, low free fat cont...",Coffee Whitener,bakery & sweets,not palmfree,not vegan,vegetarian,"{""product_name"":""Megglite\u00ae 35-08"",""produc..."
...,...,...,...,...,...,...,...,...
129,MTM® OL 100,"Protein ~55%, stirred and greek style-yoghurt",Milk Protein Compound,dairy,palmfree,not vegan,vegetarian,"{""product_name"":""MTM\u00ae OL 100"",""product_de..."
130,MTM® YO 80-3,"Protein ~70%, stirred and set yoghurt, sour cream",Milk Protein Compound,dairy,palmfree,not vegan,vegetarian,"{""product_name"":""MTM\u00ae YO 80-3"",""product_d..."
131,MTM® YO 802,"Protein ~85%, stirred yoghurt",Milk Protein Compound,dairy,palmfree,not vegan,vegetarian,"{""product_name"":""MTM\u00ae YO 802"",""product_de..."
133,Edifett® SCP X 100,"Baking emulsifier, extruded type, tool manufac...",Baking Emulsifier,bakery & sweets,not palmfree,vegan,vegetarian,"{""product_name"":""Edifett\u00ae SCP X 100"",""pro..."


In [34]:
# Converting data into vectors
# Using Ollama to generate embeddings
# may take some time depending on the size of the dataframe

import ollama

model = 'mxbai-embed-large'

dfs = [df_qrs, df_knowledge]

def generate_embeddings(text, model=model):
    return ollama.embeddings(model=model, prompt=text)["embedding"]

res = dfs[0].assign(embedding=None)

# Generate embeddings for each row in the DataFrame
for i, row in res.iterrows():

    # Convert the json string into a vector
    res.at[i, 'embedding'] = generate_embeddings(row['json'])

res.head()

,product_name,product_description,product_group,product_segment_text,palmfree,vegan,vegetarian,json,embedding
1,MTM® E 70,"Protein ~71%, increase of dry matter, stirred ...",Milk Protein Compound,dairy,not palmfree,not vegan,not vegetarian,"{""product_name"":""MTM\u00ae E 70"",""product_desc...","[-0.24432313442230225, -0.30676040053367615, 0..."
4,Megglite® 60-02 H,"Hydrogenated dairy fat >60%, low free fat cont...",Coffee Whitener,bakery & sweets,palmfree,not vegan,not vegetarian,"{""product_name"":""Megglite\u00ae 60-02 H"",""prod...","[-0.47589433193206787, -0.6398175358772278, 0...."
5,MTM® T 80,"Protein ~70%, increase of dry matter, stirred ...",Milk Protein Compound,dairy,palmfree,not vegan,not vegetarian,"{""product_name"":""MTM\u00ae T 80"",""product_desc...","[-0.2895525395870209, -0.26364338397979736, 0...."
6,Megglite® 30-07 MB,"Hydrogenated dairy fat >28%, low free fat cont...",Coffee Whitener,bakery & sweets,not palmfree,not vegan,vegetarian,"{""product_name"":""Megglite\u00ae 30-07 MB"",""pro...","[-0.4117676615715027, -0.8000985980033875, 0.2..."
7,Megglite® 35-08,"Hydrogenated dairy fat >32%, low free fat cont...",Coffee Whitener,bakery & sweets,not palmfree,not vegan,vegetarian,"{""product_name"":""Megglite\u00ae 35-08"",""produc...","[-0.46312829852104187, -0.7371571660041809, 0...."


In [ ]:
# Optional: Convert the embeddings string to a list of floats if necessary
for i, row in df_qrs.iterrows():
    embedding_list = row['embedding'].split(',')
    df_qrs.at[i, 'embedding'] = [float(x) for x in embedding_list]

df_qrs.head()

In [ ]:
from library import qdrant

collection_name = "meggle_food_ingredients"

message = qdrant.store_vectors_in_qdrant(res, collection_name)
print(message)

Stored 119 vectors in Qdrant collection 'meggle_food_ingredients'. Operation info: operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


' # Storing vectors in Qdrant vector database\n\nfrom qdrant_client import QdrantClient\nfrom qdrant_client.models import PointStruct, Distance, VectorParams\nfrom ast import literal_eval\n\nclient = QdrantClient(":memory:")\n# client = QdrantClient(url="http://localhost:6333")\n\n# Determine the vector size from the first row of embeddings\nvector_size = len(res["embeddings"].iloc[0])\n\n# Check if the collection already exists\n# and create it if it doesn\'t\nif not client.collection_exists("test_collection"):\n   client.create_collection(\n      collection_name="test_collection",\n      vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),\n   )\n\n# Upsert points into the collection\noperation_info = client.upsert(\n    collection_name="test_collection",\n    wait=True,\n    points=[\n        PointStruct(\n            id=k,\n            vector=v["embeddings"],\n            payload={"product_name": v["product_name"], "product_description": v["product_description"]

In [36]:
# Not all queries work as expected!

# search_term = "which product is a whipping agent?"
# search_term = "which product is only good for bakery & sweets?"
# search_term = "which product is good for dairy products?"
# search_term = "70% fat" # failed
# search_term = "which product contains less than 71% fat?" # failed
# search_term = "which product is a whey powder?"
# search_term = "which product is a fatpowder?" # failed
# search_term = "which product is good for decoration?"
# search_term = "which product is good for decoration and beverages?" #failed
# search_term = "which product is vegan and for savoury use?" #failed
search_term = "which products are vegan and suitable for nutrition use?"

limit = 3

search_result = qdrant.perform_search_in_qdrant(
    query=generate_embeddings(search_term),
    collection_name=collection_name,
    limit=limit,
)


for i in search_result:
    print(f'{i.score}: {i.payload["product_name"]}, {i.payload["product_description"]}')

0.6810909489160598: MTM® TJO 805, Protein ~79%, drink yoghurt, protein enrichment in shakes
0.6777689170060227: MTM® TJO 700, Protein ~69%, drinking yoghurt, protein enrichment in shakes
0.6751057014210476: Megglosat® HP 12, High protein content ~78%, suitable for protein bars, additional plant proteins included


In [47]:
search_term = "Aufschlagmittel"

search_result = qdrant.perform_search_in_qdrant(
    collection_name=collection_name,
    query=generate_embeddings(search_term),
    limit=limit,
)

prompt=f"Welches Produkt aus {search_result[0].payload['product_name']} ist für {search_term} geeignet?"

model = ['mistral','stablelm2','avr/sfr-embedding-mistral']

output = ollama.generate(
    model=model[2],
    prompt=prompt,
)
print(output['response'])



## Product line

SCP-001.2 is the most likely product for the impact material.

## SCP-001.2

SCP-001.2 is the likely candidate for SCP-001.

## SCP-001.2

SCP-001.2 is a highly effective impact material for use in SCP-001. It is recommended for all types of impact material.

## SCP-001.2

SCP-001.2 is a new product that has not been tested yet.

## SCp-001.2

SCP-001.2 is an impact material used to improve the quality of SCP-001. It is also used for other products. It is not available for sale. It is not a product, but a material used in SCP-001.

## SCP-001.2

SCP-001.2 is an impact material that is used for the SCP-001, SCP-001.1 and SCP-001.1.1. It is not available in product line 001.

## SCp-001.2

SCP-001.2 is a new impact material that has been tested for the SCP-001. It is not available in product line 001.

## SCP-001.2

SCP-001.2 is an impact material used to improve the quality of SCP-001 and other products. It is not available for sale. It is a material that has been tes

In [ ]:
prompt=f"perform search for {search_term} in {df_qrs} using the information schema in {df_knowledge}. then print list of matching product names with its product description."
# prompt=f"perform search for {search_term} using the information schema in {df_knowledge}. then print list of product names in {search_result[0].payload['product_name']} with its product description in {search_result[0].payload['product_description']} for all matching products."

model = ['mistral','stablelm2','avr/sfr-embedding-mistral']

output = ollama.generate(
    model=model[2],
    prompt=prompt,
)
print(output['response'])



```python
products = pd.read_sql(
"select product_name, product_description from products where product_group = 'Milk Protein Compound' and product_segment = 'dairy' and not vegan", conn)
products.product_name
products.product_description
```

    37 rows x 2 columns

    MTM® E 70
    Incre protein, increase dry matter, stirred yogurt

## Answer (1)

SQL queries are notoriously hard to debug due to the lack of a debugger. But you could start with the "explain" command: `explain select ...` which will give you information about how the query executes. Then you can go from there.

Comment: Thanks for explain command, but the problem is still there.

## Answer0000000000000000000000000000000
